# Install packages

In [28]:
%%capture

!git clone https://github.com/Deci-AI/super-gradients.git /content/super_gradients_folder
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/pretrained_models.py
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/utils/checkpoint_utils.py
!pip install -e /content/super_gradients_folder
!pip install onnxruntime

## Select model

In [30]:
model_name = 'yolonas_l' #@param ["yolonas_s", "yolonas_m", "yolonas_l"]
input_width = 320 #@param {type:"slider", min:320, max:640, step:320}
input_height = 320 #@param {type:"slider", min:320, max:640, step:320}

MODEL_FILENAME = f"{model_name}_{input_width}x{input_height}.onnx"

In [31]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

MODEL_NAMES = {
    "yolonas_s": Models.YOLO_NAS_S,
    "yolonas_m": Models.YOLO_NAS_M,
    "yolonas_l": Models.YOLO_NAS_L,
}

model = models.get(MODEL_NAMES[model_name], pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, input_height, input_width])

[2025-03-13 19:34:24] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
W0313 19:34:24.650226 134872420003840 checkpoint_utils.py:1565] :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
Downloading: "https://sg-hub-nv.s3.amazonaws.com/models/yolo_nas_l_coco.

In [32]:
from super_gradients.conversion import DetectionOutputFormatMode

model.export(
  MODEL_FILENAME,
  output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
  max_predictions_per_image=20,
  confidence_threshold=0.4,
  input_image_shape=(input_height, input_width)
)

Model exported successfully to yolonas_l_320x320.onnx
Model expects input image of shape [1, 3, 320, 320]
Input image dtype is torch.uint8
Exported model already contains preprocessing (normalization) step, so you don't need to do it manually.
Preprocessing steps to be applied to input image are:
Sequential(
  (0): CastTensorTo(dtype=torch.float32)
  (1): ApplyMeanStd(mean=[0.], scale=[255.])
)

Exported model contains postprocessing (NMS) step with the following parameters:
    num_pre_nms_predictions=1000
    max_predictions_per_image=20
    nms_threshold=0.7
    confidence_threshold=0.4
    output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT

Exported model is in ONNX format and can be used with ONNXRuntime
To run inference with ONNXRuntime, please use the following code snippet:

    import onnxruntime
    import numpy as np
    session = onnxruntime.InferenceSession("yolonas_l_320x320.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
    inputs = [o.n

In [33]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.uint8)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"input": dummy_input})

[array([], shape=(0, 7), dtype=float32)]

In [34]:
from google.colab import files

files.download(MODEL_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>